In [3]:
import chess.pgn
import io
from typing import List
from chess.pgn import Game

In [4]:
def load_first_chunk_of_pgn(file_path, chunk_size) -> List[Game]:
    """
    Loads the first chunk of games from a PGN file.
    
    :param file_path: Path to the PGN file.
    :param chunk_size: Number of games to load in the chunk.
    :return: List of games in the first chunk.
    """
    games: List[Game] = []
    with open(file_path) as pgn_file:
        for _ in range(chunk_size):
            try:
                game = chess.pgn.read_game(pgn_file)
                if game is None:  # End of file
                    break
                games.append(game)
            except Exception as e:
                print(f"Error reading game: {e}")
                continue
    return games

In [5]:
# Define your file path and chunk size
pgn_file_path = "dataset/lichess_db_standard_rated_2016-05.pgn"
chunk_size = 100  # for example, load the first 100 games

# Load the first chunk
first_chunk = load_first_chunk_of_pgn(pgn_file_path, chunk_size)
print(f"Loaded {len(first_chunk)} games in the first chunk")


Loaded 100 games in the first chunk


In [6]:
game = first_chunk[0]

In [12]:
vars(game.headers)

{'_tag_roster': {'Event': 'Rated Bullet tournament https://lichess.org/tournament/IaRkDsvp',
  'Site': 'https://lichess.org/r0cYFhsy',
  'Date': '????.??.??',
  'Round': '?',
  'White': 'GreatGig',
  'Black': 'hackattack',
  'Result': '0-1'},
 '_others': {'UTCDate': '2016.04.30',
  'UTCTime': '22:00:03',
  'WhiteElo': '1777',
  'BlackElo': '1809',
  'WhiteRatingDiff': '-11',
  'BlackRatingDiff': '+11',
  'ECO': 'B01',
  'Opening': 'Scandinavian Defense: Mieses-Kotroc Variation',
  'TimeControl': '60+0',
  'Termination': 'Time forfeit'}}

In [8]:
moves = [move.uci() for move in game.mainline_moves()]
print("Moves:", " ".join(moves))


Moves: e2e4 d7d5 e4d5 d8d5 b1c3 d5d8 d2d4 g8f6 g1f3 c8g4 h2h3 g4f3 g2f3 c7c6 f1g2 b8d7 c1e3 e7e6 d1d2 f6d5 c3d5 c6d5 e1c1 f8e7 c2c3 d8c7 c1b1 e8c8 f3f4 c8b8 h1g1 b8a8 g2h1 g7g6 h3h4 e7h4 f2f3 h4e7 d2c2 d7f6 h1g2 f6h5 g2h3 h5f4 e3f4 c7f4 d1f1 f4d6 g1g4 d8f8 f1g1 f7f5 g4g2 e7f6 g2g3 f8g8 h3f1 g8g7 f1d3 h8g8 c2h2 d6b8 h2g2 b8c8 f3f4 c8c6 g2f2 f6h4 g3g6 h4f2 g6g7 g8g7 g1g7 a7a6 g7g8 a8a7 g8h8 c6d7 h8h7 d7h7


In [9]:
game.headers["Result"]

'0-1'